***GENERATED CODE FOR bnpparibascardiffclaimsmanagement PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v3', 'transformation_label': 'String Indexer'}], 'feature': 'v3', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'C', 'max': 'C', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v3'}, {'feature_label': 'v3', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v3')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v22', 'transformation_label': 'String Indexer'}], 'feature': 'v22', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AACV', 'max': 'ZYN', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v22'}, {'feature_label': 'v22', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v22')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v24', 'transformation_label': 'String Indexer'}], 'feature': 'v24', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'E', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v24'}, {'feature_label': 'v24', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v24')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v30', 'transformation_label': 'String Indexer'}], 'feature': 'v30', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'G', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v30'}, {'feature_label': 'v30', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v30')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v31', 'transformation_label': 'String Indexer'}], 'feature': 'v31', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'C', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v31'}, {'feature_label': 'v31', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v31')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v47', 'transformation_label': 'String Indexer'}], 'feature': 'v47', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'C', 'max': 'J', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v47'}, {'feature_label': 'v47', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v47')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v52', 'transformation_label': 'String Indexer'}], 'feature': 'v52', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'L', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v52'}, {'feature_label': 'v52', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v52')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v56', 'transformation_label': 'String Indexer'}], 'feature': 'v56', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AF', 'max': 'Y', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v56'}, {'feature_label': 'v56', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v56')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v66', 'transformation_label': 'String Indexer'}], 'feature': 'v66', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'C', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v66'}, {'feature_label': 'v66', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v66')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v71', 'transformation_label': 'String Indexer'}], 'feature': 'v71', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'B', 'max': 'F', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v71'}, {'feature_label': 'v71', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v71')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v74', 'transformation_label': 'String Indexer'}], 'feature': 'v74', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'B', 'max': 'C', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v74'}, {'feature_label': 'v74', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v74')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v75', 'transformation_label': 'String Indexer'}], 'feature': 'v75', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'B', 'max': 'D', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v75'}, {'feature_label': 'v75', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v75')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v79', 'transformation_label': 'String Indexer'}], 'feature': 'v79', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'B', 'max': 'Q', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v79'}, {'feature_label': 'v79', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v79')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v91', 'transformation_label': 'String Indexer'}], 'feature': 'v91', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'G', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v91'}, {'feature_label': 'v91', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v91')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v107', 'transformation_label': 'String Indexer'}], 'feature': 'v107', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'E', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v107'}, {'feature_label': 'v107', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v107')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v110', 'transformation_label': 'String Indexer'}], 'feature': 'v110', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'B', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v110'}, {'feature_label': 'v110', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v110')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v112', 'transformation_label': 'String Indexer'}], 'feature': 'v112', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'V', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v112'}, {'feature_label': 'v112', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v112')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v113', 'transformation_label': 'String Indexer'}], 'feature': 'v113', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'Z', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v113'}, {'feature_label': 'v113', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v113')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'v125', 'transformation_label': 'String Indexer'}], 'feature': 'v125', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'Z', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'v125'}, {'feature_label': 'v125', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('v125')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run bnpparibascardiffclaimsmanagementHooks.ipynb
try:
	#sourcePreExecutionHook()

	train = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1623091484874_train.csv', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'delimiter': ',', 'is_header': 'Use Header Line', 'FilePath': '/Healthcare Models/RCM - Claims Management - Classification/train.csv', 'filename': '1623091484874_train.csv', 'viewFileName': 'train.csv', 'results_url': 'http://3.140.115.35:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run bnpparibascardiffclaimsmanagementHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(train,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "ID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "141343.8", "stddev": "56336.7", "min": "60183", "max": "228654", "missing": "0"}, "updatedLabel": "ID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "target", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.67", "stddev": "0.47", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "target"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.65", "stddev": "1.08", "min": "-9.37314662282E-7", "max": "6.92513415148", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "7.74", "stddev": "3.07", "min": "1.53234983378", "max": "18.356968502", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v2"}, {"transformationsData": [{"feature_label": "v3", "transformation_label": "String Indexer"}], "feature": "v3", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "C", "max": "C", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v4", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "4.22", "stddev": "1.11", "min": "1.0312788102", "max": "8.62833062799", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v5", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "8.61", "stddev": "2.07", "min": "2.48685499175", "max": "17.6529932848", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v5"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v6", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.43", "stddev": "0.62", "min": "0.43010675622", "max": "6.06577917532", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v6"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v7", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.53", "stddev": "0.58", "min": "1.06712491469", "max": "4.90988147209", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v7"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v8", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.09", "stddev": "2.15", "min": "0.00431501827804", "max": "20.000000997", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v8"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v9", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "9.18", "stddev": "1.76", "min": "3.72092934477", "max": "17.1428579321", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v9"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v10", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.51", "stddev": "0.83", "min": "0.0437636908306", "max": "6.54266993309", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v10"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v11", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "15.49", "stddev": "0.75", "min": "12.7924523574", "max": "17.2881360423", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v11"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v12", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "6.62", "stddev": "0.52", "min": "3.89049245556", "max": "8.74224388067", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v12"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v13", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.59", "stddev": "1.02", "min": "1.46650333309", "max": "10.6675221412", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v13"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v14", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "11.67", "stddev": "0.96", "min": "7.29590781497", "max": "14.2634227606", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v14"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v15", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.04", "stddev": "0.7", "min": "0.496942180317", "max": "5.29026172337", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v15"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v16", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "5.02", "stddev": "1.79", "min": "-9.90145408605E-7", "max": "20.000000401", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v16"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v17", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.76", "stddev": "1.81", "min": "0.631630272585", "max": "16.7105082513", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v17"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v18", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.75", "stddev": "0.55", "min": "0.0289771185384", "max": "6.20679997008", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v18"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v19", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.21", "stddev": "0.07", "min": "0.118675019098", "max": "0.851914037347", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v19"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v20", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "17.77", "stddev": "1.08", "min": "12.11266316", "max": "19.6678120024", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v20"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v21", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "6.83", "stddev": "0.86", "min": "3.38559494027", "max": "11.3931859311", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v21"}, {"transformationsData": [{"feature_label": "v22", "transformation_label": "String Indexer"}], "feature": "v22", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AACV", "max": "ZYN", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v22"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v23", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.72", "stddev": "3.21", "min": "-9.97899839693E-7", "max": "20.0000009951", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v23"}, {"transformationsData": [{"feature_label": "v24", "transformation_label": "String Indexer"}], "feature": "v24", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "E", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v24"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v25", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.53", "stddev": "2.7", "min": "0.0473596843287", "max": "20.000000533", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v25"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v26", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.87", "stddev": "0.54", "min": "0.621117953574", "max": "3.98884287713", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v26"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v27", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.85", "stddev": "0.81", "min": "0.665519861737", "max": "5.44488775078", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v27"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v28", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "4.64", "stddev": "2.58", "min": "0.213812250944", "max": "12.6683761679", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v28"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v29", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "8.17", "stddev": "1.19", "min": "3.60773938323", "max": "11.0355648234", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v29"}, {"transformationsData": [{"feature_label": "v30", "transformation_label": "String Indexer"}], "feature": "v30", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "G", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v30"}, {"transformationsData": [{"feature_label": "v31", "transformation_label": "String Indexer"}], "feature": "v31", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "C", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v31"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v32", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.61", "stddev": "0.52", "min": "0.35503048764", "max": "4.04979018267", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v32"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v33", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.07", "stddev": "0.89", "min": "0.508473725403", "max": "5.99097157639", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v33"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v34", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "6.23", "stddev": "2.04", "min": "1.63859227122", "max": "11.2190439401", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v34"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v35", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "8.09", "stddev": "1.25", "min": "3.97210319634", "max": "11.5654651829", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v35"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v36", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "13.14", "stddev": "2.31", "min": "6.03212278831", "max": "17.0040263087", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v36"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v37", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.72", "stddev": "0.49", "min": "-9.22469302653E-7", "max": "3.44000042578", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v37"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v38", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.03", "stddev": "0.45", "min": "0", "max": "9", "missing": "0"}, "updatedLabel": "v38"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v39", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.31", "stddev": "2.3", "min": "-9.97041738059E-7", "max": "16.243481159", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v39"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v40", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "10.17", "stddev": "3.31", "min": "1.8710655617", "max": "17.5153755493", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v40"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v41", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "7.14", "stddev": "0.93", "min": "3.92982525109", "max": "9.81310897824", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v41"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v42", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "12.79", "stddev": "0.93", "min": "8.60843507386", "max": "15.9199078973", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v42"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v43", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.22", "stddev": "0.62", "min": "0.408162584313", "max": "4.74576244194", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v43"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v44", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "10.72", "stddev": "2.1", "min": "3.41157708744", "max": "16.1644727085", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v44"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v45", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "9.27", "stddev": "1.94", "min": "3.79449242108", "max": "15.4471727649", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v45"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v46", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.5", "stddev": "2.64", "min": "0.0739233603765", "max": "20.0000003477", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v46"}, {"transformationsData": [{"feature_label": "v47", "transformation_label": "String Indexer"}], "feature": "v47", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "C", "max": "J", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v47"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v48", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "12.45", "stddev": "2.04", "min": "5.28630919854", "max": "18.4169728635", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v48"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v49", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "7.94", "stddev": "0.82", "min": "4.59647883328", "max": "10.4083575488", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v49"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v50", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.4", "stddev": "1.25", "min": "0.0425508459716", "max": "13.6571054675", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v50"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v51", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "7.15", "stddev": "2.5", "min": "1.146772473", "max": "18.9492371411", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v51"}, {"transformationsData": [{"feature_label": "v52", "transformation_label": "String Indexer"}], "feature": "v52", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "L", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v52"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v53", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "15.74", "stddev": "0.73", "min": "13.3235723549", "max": "17.4999993199", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v53"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v54", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.98", "stddev": "1.88", "min": "0.0164805206827", "max": "20.000000238", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v54"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v55", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.52", "stddev": "0.81", "min": "0.254236466656", "max": "4.94094567197", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v55"}, {"transformationsData": [{"feature_label": "v56", "transformation_label": "String Indexer"}], "feature": "v56", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AF", "max": "Y", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v56"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v57", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "4.13", "stddev": "0.65", "min": "2.18309829574", "max": "6.17541833395", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v57"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v58", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "6.17", "stddev": "6.27", "min": "-8.99582296404E-7", "max": "19.9829492562", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v58"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v59", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "10.7", "stddev": "2.1", "min": "4.75041488029", "max": "16.5469214517", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v59"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v60", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.69", "stddev": "0.5", "min": "0.230608618213", "max": "3.36531798181", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v60"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v61", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "14.77", "stddev": "2.09", "min": "6.61744606409", "max": "17.9987960237", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v61"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v62", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.25", "stddev": "0.66", "min": "0", "max": "6", "missing": "0"}, "updatedLabel": "v62"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v63", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.4", "stddev": "2.78", "min": "0.0635046155099", "max": "20.0000007239", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v63"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v64", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "6.28", "stddev": "2.43", "min": "1.01712374184", "max": "17.8897184011", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v64"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v65", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "15.96", "stddev": "1.82", "min": "6.28246161951", "max": "18.8170285143", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v65"}, {"transformationsData": [{"feature_label": "v66", "transformation_label": "String Indexer"}], "feature": "v66", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "C", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v66"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v67", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "9.16", "stddev": "1.03", "min": "5.49707635778", "max": "12.0505392384", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v67"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v68", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "17.66", "stddev": "2.17", "min": "4.67774913113", "max": "19.9999996912", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v68"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v69", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "9.49", "stddev": "1.89", "min": "1.12199163068E-7", "max": "19.9999994405", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v69"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v70", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "12.12", "stddev": "2.42", "min": "1.65284159892", "max": "17.4909359197", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v70"}, {"transformationsData": [{"feature_label": "v71", "transformation_label": "String Indexer"}], "feature": "v71", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "B", "max": "F", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v71"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v72", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.29", "stddev": "0.71", "min": "1", "max": "9", "missing": "0"}, "updatedLabel": "v72"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v73", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.37", "stddev": "0.79", "min": "0.62761452489", "max": "5.66037731059", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v73"}, {"transformationsData": [{"feature_label": "v74", "transformation_label": "String Indexer"}], "feature": "v74", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "B", "max": "C", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v74"}, {"transformationsData": [{"feature_label": "v75", "transformation_label": "String Indexer"}], "feature": "v75", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "B", "max": "D", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v75"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v76", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.35", "stddev": "1.42", "min": "0.296282691815", "max": "14.1877822444", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v76"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v77", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "7.28", "stddev": "1.2", "min": "3.16838285859", "max": "10.3814429465", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v77"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v78", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "13.13", "stddev": "1.75", "min": "5.45454616955", "max": "19.999999974", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v78"}, {"transformationsData": [{"feature_label": "v79", "transformation_label": "String Indexer"}], "feature": "v79", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "B", "max": "Q", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v79"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v80", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.25", "stddev": "1.08", "min": "-5.7426790645E-7", "max": "6.66666701622", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v80"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v81", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "7.26", "stddev": "2.29", "min": "1.17498926934", "max": "15.426967475", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v81"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v82", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "6.73", "stddev": "3.73", "min": "1.00156460071", "max": "16.2638645756", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v82"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v83", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.07", "stddev": "1.0", "min": "0.221402597175", "max": "6.47517455942", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v83"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v84", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.58", "stddev": "0.94", "min": "-9.99424467098E-7", "max": "6.18556796446", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v84"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v85", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.72", "stddev": "1.43", "min": "-6.91566338305E-7", "max": "8.05633895891", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v85"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v86", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.17", "stddev": "0.41", "min": "0.238095089573", "max": "3.29441790237", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v86"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v87", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "10.01", "stddev": "2.98", "min": "1.67039405692", "max": "18.5733875308", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v87"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v88", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.03", "stddev": "1.04", "min": "0.213789517012", "max": "7.92452864516", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v88"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v89", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.04", "stddev": "2.41", "min": "0.0251830852679", "max": "20.0000009093", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v89"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v90", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.98", "stddev": "0.16", "min": "0.517877404139", "max": "1.65106084775", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v90"}, {"transformationsData": [{"feature_label": "v91", "transformation_label": "String Indexer"}], "feature": "v91", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "G", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v91"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v92", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.56", "stddev": "0.19", "min": "0.122996173589", "max": "1.5913069214", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v92"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v93", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "5.36", "stddev": "1.55", "min": "0.846780272481", "max": "14.1848930646", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v93"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v94", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.81", "stddev": "0.76", "min": "2.18262749476", "max": "8.97576110861", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v94"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v95", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.64", "stddev": "0.21", "min": "0.171329438627", "max": "2.0394910877", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v95"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v96", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "6.44", "stddev": "1.09", "min": "2.08785494515", "max": "9.00763320434", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v96"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v97", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "7.54", "stddev": "1.78", "min": "1.37931069905", "max": "14.5126352917", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v97"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v98", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "7.89", "stddev": "2.29", "min": "0.39850029831", "max": "16.3741866416", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v98"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v99", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.21", "stddev": "0.43", "min": "-1.52475254208E-7", "max": "2.39306854562", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v99"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v100", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "13.28", "stddev": "6.47", "min": "-9.48833779844E-7", "max": "20.0000009557", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v100"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v101", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "6.8", "stddev": "2.3", "min": "1.61107135089", "max": "14.6740629727", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v101"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v102", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.74", "stddev": "1.6", "min": "0.548797321788", "max": "15.8561984265", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v102"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v103", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "5.19", "stddev": "1.16", "min": "2.56571612272", "max": "9.97923733952", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v103"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v104", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.54", "stddev": "0.78", "min": "1.0283237126", "max": "8.70253125661", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v104"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v105", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.8", "stddev": "1.8", "min": "0.00155981667375", "max": "13.0496539053", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v105"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v106", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "11.79", "stddev": "2.82", "min": "3.39460869465", "max": "18.0976678943", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v106"}, {"transformationsData": [{"feature_label": "v107", "transformation_label": "String Indexer"}], "feature": "v107", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "E", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v107"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v108", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.17", "stddev": "0.93", "min": "0.485611233405", "max": "9.49180378051", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v108"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v109", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.92", "stddev": "3.44", "min": "-8.15660382946E-7", "max": "20.0000005734", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v109"}, {"transformationsData": [{"feature_label": "v110", "transformation_label": "String Indexer"}], "feature": "v110", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "B", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v110"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v111", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.27", "stddev": "1.43", "min": "0.64516060036", "max": "10.7838628122", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v111"}, {"transformationsData": [{"feature_label": "v112", "transformation_label": "String Indexer"}], "feature": "v112", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "V", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v112"}, {"transformationsData": [{"feature_label": "v113", "transformation_label": "String Indexer"}], "feature": "v113", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "Z", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v113"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v114", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "13.65", "stddev": "2.55", "min": "6.13470797341", "max": "18.9645247764", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v114"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v115", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "10.54", "stddev": "1.79", "min": "4.58715536729", "max": "20.0000007722", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v115"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v116", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.3", "stddev": "0.68", "min": "0.755940089688", "max": "4.78952231894", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v116"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v117", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "7.94", "stddev": "3.37", "min": "-7.17774843622E-7", "max": "16.729949554", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v117"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v118", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "8.16", "stddev": "1.82", "min": "2.06896536826", "max": "15.0180505016", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v118"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v119", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.03", "stddev": "4.28", "min": "-9.90244036456E-7", "max": "19.2424892229", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v119"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v120", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.28", "stddev": "0.73", "min": "8.44247154348E-7", "max": "4.53278174306", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v120"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v121", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.68", "stddev": "1.38", "min": "0.428570797854", "max": "10.8188593431", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v121"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v122", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "6.94", "stddev": "1.73", "min": "1.96969759191", "max": "13.3333328069", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v122"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v123", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.21", "stddev": "2.35", "min": "0.372113726847", "max": "16.2407137241", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v123"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v124", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.78", "stddev": "1.94", "min": "-7.49548780193E-7", "max": "19.9999991276", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v124"}, {"transformationsData": [{"feature_label": "v125", "transformation_label": "String Indexer"}], "feature": "v125", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "Z", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "v125"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v126", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.68", "stddev": "0.44", "min": "0.923436677215", "max": "6.88554267855", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v126"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v127", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.33", "stddev": "1.77", "min": "0.25406422015", "max": "13.1766850016", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v127"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v128", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.03", "stddev": "1.08", "min": "0.463262010501", "max": "12.7216864617", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v128"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v129", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.05", "stddev": "0.32", "min": "0", "max": "4", "missing": "0"}, "updatedLabel": "v129"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v130", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.82", "stddev": "1.25", "min": "0.316831688988", "max": "12.7304354628", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v130"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "v131", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.74", "stddev": "1.07", "min": "-9.96405448508E-7", "max": "7.14285775328", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "v131"}, {"feature": "v3_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "v3_stringindexer"}, {"feature": "v22_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "273.63", "stddev": "165.71", "min": "0.0", "max": "446.0", "missing": "0"}, "updatedLabel": "v22_stringindexer"}, {"feature": "v24_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "1.19", "stddev": "1.18", "min": "0.0", "max": "4.0", "missing": "0"}, "updatedLabel": "v24_stringindexer"}, {"feature": "v30_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "0.88", "stddev": "1.29", "min": "0.0", "max": "5.0", "missing": "0"}, "updatedLabel": "v30_stringindexer"}, {"feature": "v31_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.03", "stddev": "0.18", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "v31_stringindexer"}, {"feature": "v47_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "0.98", "stddev": "1.5", "min": "0.0", "max": "5.0", "missing": "0"}, "updatedLabel": "v47_stringindexer"}, {"feature": "v52_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "5.06", "stddev": "3.5", "min": "0.0", "max": "11.0", "missing": "0"}, "updatedLabel": "v52_stringindexer"}, {"feature": "v56_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "6.28", "stddev": "7.1", "min": "0.0", "max": "34.0", "missing": "0"}, "updatedLabel": "v56_stringindexer"}, {"feature": "v66_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.58", "stddev": "0.78", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "v66_stringindexer"}, {"feature": "v71_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.42", "stddev": "0.68", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "v71_stringindexer"}, {"feature": "v74_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.04", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "v74_stringindexer"}, {"feature": "v75_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.3", "stddev": "0.46", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "v75_stringindexer"}, {"feature": "v79_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "1.66", "stddev": "2.02", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "v79_stringindexer"}, {"feature": "v91_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "1.84", "stddev": "1.39", "min": "0.0", "max": "5.0", "missing": "0"}, "updatedLabel": "v91_stringindexer"}, {"feature": "v107_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "1.88", "stddev": "1.4", "min": "0.0", "max": "5.0", "missing": "0"}, "updatedLabel": "v107_stringindexer"}, {"feature": "v110_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.08", "stddev": "0.27", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "v110_stringindexer"}, {"feature": "v112_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "6.5", "stddev": "5.9", "min": "0.0", "max": "21.0", "missing": "0"}, "updatedLabel": "v112_stringindexer"}, {"feature": "v113_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "5.5", "stddev": "6.6", "min": "0.0", "max": "29.0", "missing": "0"}, "updatedLabel": "v113_stringindexer"}, {"feature": "v125_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "21.89", "stddev": "20.33", "min": "0.0", "max": "80.0", "missing": "0"}, "updatedLabel": "v125_stringindexer"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run bnpparibascardiffclaimsmanagementHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["ID", "v1", "v2", "v3_stringindexer", "v4", "v5", "v6", "v7", "v8", "v9", "v10", "v11", "v12", "v13", "v14", "v15", "v16", "v17", "v18", "v19", "v20", "v21", "v22_stringindexer", "v23", "v24_stringindexer", "v25", "v26", "v27", "v28", "v29", "v30_stringindexer", "v31_stringindexer", "v32", "v33", "v34", "v35", "v36", "v37", "v38", "v39", "v40", "v41", "v42", "v43", "v44", "v45", "v46", "v47_stringindexer", "v48", "v49", "v50", "v51", "v52_stringindexer", "v53", "v54", "v55", "v56_stringindexer", "v57", "v58", "v59", "v60", "v61", "v62", "v63", "v64", "v65", "v66_stringindexer", "v67", "v68", "v69", "v70", "v71_stringindexer", "v72", "v73", "v74_stringindexer", "v75_stringindexer", "v76", "v77", "v78", "v79_stringindexer", "v80", "v81", "v82", "v83", "v84", "v85", "v86", "v87", "v88", "v89", "v90", "v91_stringindexer", "v92", "v93", "v94", "v95", "v96", "v97", "v98", "v99", "v100", "v101", "v102", "v103", "v104", "v105", "v106", "v107_stringindexer", "v108", "v109", "v110_stringindexer", "v111", "v112_stringindexer", "v113_stringindexer", "v114", "v115", "v116", "v117", "v118", "v119", "v120", "v121", "v122", "v123", "v124", "v125_stringindexer", "v126", "v127", "v128", "v129", "v130", "v131", "v3_stringindexer", "v22_stringindexer", "v24_stringindexer", "v30_stringindexer", "v31_stringindexer", "v47_stringindexer", "v52_stringindexer", "v56_stringindexer", "v66_stringindexer", "v71_stringindexer", "v74_stringindexer", "v75_stringindexer", "v79_stringindexer", "v91_stringindexer", "v107_stringindexer", "v110_stringindexer", "v112_stringindexer", "v113_stringindexer", "v125_stringindexer"], "target")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

